In [11]:
import time
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss


def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt


def convert_data(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    user_query_day = data.groupby(['user_id', 'day']).size(
    ).reset_index().rename(columns={0: 'user_query_day'})
    data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
    user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index().rename(
        columns={0: 'user_query_day_hour'})
    data = pd.merge(data, user_query_day_hour, 'left',
                    on=['user_id', 'day', 'hour'])

    return data

In [17]:
data = pd.read_csv('round1_ijcai_18_train_20180301.txt', sep=' ')
data.drop_duplicates(inplace=True)
data = convert_data(data)


train = data.loc[data.day < 24]  # 18,19,20,21,22,23,24
test = data.loc[data.day == 24]  # 暂时先使用第24天作为验证集


features = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
            'item_collected_level', 'item_pv_level', 'user_gender_id', 'user_occupation_id',
            'user_age_level', 'user_star_level', 
            'context_page_id', 'hour', 'shop_id', 'shop_review_num_level', 'shop_star_level',
            'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
            'user_query_day', 'user_query_day_hour'
            ]

target = 'is_trade'

clf = lgb.LGBMClassifier(
    num_leaves=63, max_depth=7, n_estimators=80, n_jobs=20)
clf.fit(train[features], train[target], feature_name=features,
        categorical_feature=['user_gender_id'])
lgb_predict = clf.predict_proba(test[features],)[:, 1]
print(log_loss(test[target], lgb_predict))


/home/user02/miniconda3/envs/gluon/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['item_brand_id']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


0.0819597542973
